In [ ]:
import os, sys
import timeit

import numpy as np
import pandas as pd
import datetime
import math

import string

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from google.colab import drive
drive.mount('/content/drive',force_remount=True)

import tensorflow as tf
from tensorflow import keras

from sklearn.preprocessing import OneHotEncoder,MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, KFold
from sklearn.metrics import *
from sklearn.utils import class_weight

from xgboost.sklearn import XGBClassifier
! pip install -q catboost
from catboost import CatBoostClassifier

import pickle,joblib

import warnings
warnings.filterwarnings(action='ignore')

Mounted at /content/drive


In [ ]:
def preprocessing_1(df):

  # 1) D / H / L 코드 파생변수 생성  
  D_code_1=D_code.rename(columns={'속성 D 코드':'person_prefer_d_1','속성 D 세분류코드':'person_prefer_d_1_m','속성 D 대분류코드':'person_prefer_d_1_l'}, inplace = False).drop(['속성 D 소분류코드','속성 D 중분류코드'],axis="columns")
  D_code_2=D_code.rename(columns={'속성 D 코드':'person_prefer_d_2','속성 D 세분류코드':'person_prefer_d_2_m','속성 D 대분류코드':'person_prefer_d_2_l'}, inplace = False).drop(['속성 D 소분류코드','속성 D 중분류코드'],axis="columns")
  D_code_3=D_code.rename(columns={'속성 D 코드':'person_prefer_d_3','속성 D 세분류코드':'person_prefer_d_3_m','속성 D 대분류코드':'person_prefer_d_3_l'}, inplace = False).drop(['속성 D 소분류코드','속성 D 중분류코드'],axis="columns")
  D_code_contents=D_code.rename(columns = {'속성 D 코드' : 'contents_attribute_d','속성 D 세분류코드':'contents_attribute_d_m','속성 D 대분류코드':'contents_attribute_d_l'}, inplace = False).drop(['속성 D 소분류코드','속성 D 중분류코드'],axis="columns")
  H_code_1=H_code.rename(columns={'속성 H 코드':'person_prefer_h_1','속성 H 중분류코드':'person_prefer_h_1_m','속성 H 대분류코드':'person_prefer_h_1_l'}, inplace = False)
  H_code_2=H_code.rename(columns={'속성 H 코드':'person_prefer_h_2','속성 H 중분류코드':'person_prefer_h_2_m','속성 H 대분류코드':'person_prefer_h_2_l'}, inplace = False)
  H_code_3=H_code.rename(columns={'속성 H 코드':'person_prefer_h_3','속성 H 중분류코드':'person_prefer_h_3_m','속성 H 대분류코드':'person_prefer_h_3_l'}, inplace = False)
  H_code_contents=H_code.rename(columns={'속성 H 코드' : 'contents_attribute_h','속성 H 중분류코드':'contents_attribute_h_m','속성 H 대분류코드':'contents_attribute_h_l'}, inplace = False)
  L_code_contents=L_code.rename(columns={'속성 L 코드' : 'contents_attribute_l','속성 L 세분류코드':'contents_attribute_l_m','속성 L 대분류코드':'contents_attribute_l_l'}, inplace = False).drop(['속성 L 소분류코드','속성 L 중분류코드'],axis="columns")

  df_list=[D_code_1,D_code_2,D_code_3,D_code_contents,
          H_code_1,H_code_2,H_code_3,H_code_contents, L_code_contents]
  df_column=["person_prefer_d_1","person_prefer_d_2","person_prefer_d_3","contents_attribute_d",
            "person_prefer_h_1","person_prefer_h_2","person_prefer_h_3","contents_attribute_h",
            "contents_attribute_l"]

  for i in range(0,len(df_column)):
    df = pd.merge(df, df_list[i],on=df_column[i]) 

  df.loc[(df['person_prefer_d_1_m']==df['contents_attribute_d_m'])|(df['person_prefer_d_2_m']==df['contents_attribute_d_m'])| (df['person_prefer_d_3_m']==df['contents_attribute_d_m']),'d_m_match_123']=True
  df.loc[(df['person_prefer_d_1_l']==df['contents_attribute_d_l'])|(df['person_prefer_d_2_l']==df['contents_attribute_d_l'])| (df['person_prefer_d_3_l']==df['contents_attribute_d_l']),'d_l_match_123']=True
  df.loc[(df['person_prefer_d_1']==df['contents_attribute_d'])|(df['person_prefer_d_2']==df['contents_attribute_d'])| (df['person_prefer_d_3']==df['contents_attribute_d']),'d_s_match_123']=True
  df.loc[(df['person_prefer_h_1_l']==df['contents_attribute_h_l'])|(df['person_prefer_h_2_l']==df['contents_attribute_h_l'])| (df['person_prefer_h_3_l']==df['contents_attribute_h_l']),'h_l_match_123']=True
  df.loc[(df['person_prefer_h_1_m']==df['contents_attribute_h_m'])|(df['person_prefer_h_2_m']==df['contents_attribute_h_m'])| (df['person_prefer_h_3_m']==df['contents_attribute_h_m']),'h_m_match_123']=True
  df.loc[(df['person_prefer_h_1']==df['contents_attribute_h'])|(df['person_prefer_h_2']==df['contents_attribute_h'])| (df['person_prefer_h_3']==df['contents_attribute_h']),'h_s_match_123']=True
  
  df = df.fillna(False).sort_values(by=['id'],axis=0) 


  # 2) 변수형 변환
  df.loc[:, df.columns != 'contents_open_dt'] = \
      df.loc[:, df.columns != 'contents_open_dt'].astype('int') # boolean -> int
  numeric_columns = ['contents_rn','person_rn','contents_attribute_j'
                     ]
  categorical_columns = list(df.columns.drop(numeric_columns))
  df=df[categorical_columns].astype('category')

  # 3) id list 저장 
  id_list = list(df['id'])
  df = df.sort_values(by=['id'],axis=0).set_index('id')


  # 4) 불필요한 컬럼 삭제 
  df.drop(['person_prefer_f','person_prefer_g','contents_open_dt'],
          axis="columns",inplace=True)
  
  return id_list, df  


In [ ]:
# 디렉토리 설정: "[DACON] 잡케어 추천 알고리즘 경진대회" 폴더로 이동
path = "/content/drive/MyDrive/"
os.chdir(path)

# 파일 불러오기 
train = pd.read_csv('Jobcare_data/train.csv')
test = pd.read_csv('Jobcare_data/test.csv')
D_code=pd.read_csv('Jobcare_data/속성_D_코드.csv')
L_code=pd.read_csv('Jobcare_data/속성_L_코드.csv')
H_code=pd.read_csv('Jobcare_data/속성_H_코드.csv')

# 1타입 전처리 함수 적용하기 
train1_idx, train1 = preprocessing_1(train)
#train1.to_csv('train1.csv', header=True,index=False)

test1_idx, test1 = preprocessing_1(test)  
#test1.to_csv('test1.csv', header=True,index=False)